<h1><font color="#008800">Python aplicado a la Gestión de Carteras</font></h1>

# DESCARGA DE DATOS

In [1]:
"""
Estas dos líneas ayudan a mejorar la visualización de las gráficas generadas con Matplotlib 
en un Jupyter Notebook, mostrándolas directamente en el cuaderno y optimizando su calidad 
para pantallas de alta resolución
"""
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

# print(plt.style.available) #list of available styles
plt.style.use('seaborn-v0_8')

plt.rcParams['figure.figsize'] = [16, 9]
plt.rcParams['figure.dpi'] = 100
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
import nasdaqdatalink
nasdaqdatalink.read_key(filename="credential/nasdaq_key")

## Descargarse datos financieros

#### 1. Descargarse datos de Yahoo Finance

`yfinance` es una librería de Python que proporciona una interfaz sencilla para acceder y descargar datos financieros de Yahoo Finance. Esta biblioteca facilita la obtención de información detallada sobre precios históricos, datos de acciones, divisas, bonos, índices, etc. Esta librería es directamente compatible con `pandas`, ya que devuelve los datos en formato DataFrame.

Para instalar la libería si fuese necesario:

```pip install yfinance```

In [4]:
import yfinance as yf

In [5]:
# Descargamos los datos de Apple ('AAPL') entre los años 2000 y 2010

data_yahoo = yf.download('AAPL', 
                         start='2000-01-01', # Se indica con el argumento 'start' una fecha de incio (incluida)
                         end='2011-01-01', # Se indica con el argumento 'end' una fecha fin (excluida)
                         progress=False) # El argumento progress = False desactiva la barra de descarga (verbose)

La variable `data_yahoo` es un DataFrame de Pandas con la siguiente información:

* Como índice del DataFrame, **Date**: La fecha a la que se recoge la información
* **Open**: Precio de apertura del activo
* **High**: Precio máximo alcanzado por el activo
* **Low**: Precio mínimo alcanzado por el activo
* **Close**: Precio de cierre del activo
* **Adj Close**: Precio de cierre ajustado por eventos corporativos, como dividendos, splits, etc.
* **Volume**: Número total de transacciones negociadas

Veamos el aspecto que tiene:

In [6]:
data_yahoo.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-01-03,0.936384,1.004464,0.907924,0.999442,0.847207,535796800
2000-01-04,0.966518,0.987723,0.903460,0.915179,0.775779,512377600
2000-01-05,0.926339,0.987165,0.919643,0.928571,0.787131,778321600
2000-01-06,0.947545,0.955357,0.848214,0.848214,0.719014,767972800
2000-01-07,0.861607,0.901786,0.852679,0.888393,0.753073,460734400


In [7]:
data_yahoo.tail() # Incluye el 31 de diciembre de 2010

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-12-27,11.530357,11.622857,11.482857,11.595714,9.829451,249816000
2010-12-28,11.639643,11.666429,11.609286,11.623929,9.853371,175924000
2010-12-29,11.650714,11.658929,11.610714,11.617500,9.847922,163139200
2010-12-30,11.624286,11.625357,11.537500,11.559286,9.798574,157494400
2010-12-31,11.533929,11.552857,11.475357,11.520000,9.765269,193508000


In [8]:
data_yahoo.shape # 2767 filas, 6 columnas

(2767, 6)

Lo más común es trabajar con los precios de cierre. Concretamente, vamos a seleccionar la columna de precios ajustados `Adj Close`.

**OBSERVACIÓN:** Hay que tener cuidado con las transformaciones que hacemos sobre el DataFrame. Ojo si lo que seleccionamos es un DataFrame o una Serie.

In [9]:
# Si elijo una sola columna, me devuelve una serie
serie_yahoo = data_yahoo['Adj Close']
serie_yahoo.head()

Date
2000-01-03    0.847207
2000-01-04    0.775779
2000-01-05    0.787131
2000-01-06    0.719014
2000-01-07    0.753073
Name: Adj Close, dtype: float64

In [10]:
# Si elijo una lista de columnas (aunque solo haya una), me devuelve un dataframe con una sola columna
df1_yahoo = data_yahoo[['Adj Close']]
df1_yahoo.head()

,Adj Close
Date,
2000-01-03,0.847207
2000-01-04,0.775779
2000-01-05,0.787131
2000-01-06,0.719014
2000-01-07,0.753073


En la celda a continuación, descargamos los datos de varios activos financieros a la vez. Simplemente pasamos a `yf.download` una lista de activos en lugar de solo uno:

In [11]:
activos = ['AAPL', 'MSFT', 'AMZN', 'GOOG', 'TSLA', 'NVDA', 'ADBE', 'NFLX']
df_yahoo = yf.download(activos, 
                       start='2000-01-01', 
                       end='2011-01-01',
                       progress=False)

En este caso, al descargar una lista de activos, yfinance nos devuelve un DataFrame multinivel en las columnas `(atributo, activo)`, donde el primer nivel representa los atributos (open, high, low, close, etc.) y el segundo nivel el activo correspondiente (AAPL, MSFT, etc.):


In [12]:
df_yahoo.head()

Adj Close                                                           \
                AAPL       ADBE      AMZN GOOG       MSFT NFLX      NVDA TSLA   
Date                                                                            
2000-01-03  0.847207  16.274672  4.468750  NaN  36.132278  NaN  0.894650  NaN   
2000-01-04  0.775779  14.909401  4.096875  NaN  34.911705  NaN  0.870762  NaN   
2000-01-05  0.787131  15.204173  3.487500  NaN  35.279808  NaN  0.842094  NaN   
2000-01-06  0.719014  15.328292  3.278125  NaN  34.098019  NaN  0.787149  NaN   
2000-01-07  0.753073  16.072983  3.478125  NaN  34.543610  NaN  0.800288  NaN   

               Close             ...      Open          Volume            \
                AAPL       ADBE  ...      NVDA TSLA       AAPL      ADBE   
Date                             ...                                       
2000-01-03  0.999442  16.390625  ...  0.984375  NaN  535796800   7384400   
2000-01-04  0.915179  15.015625  ...  0.958333  NaN  512377600   7813200   
2000-01-05  0.928571  15.312500  ...  0.921875  NaN  778321600  14927200   
2000-01-06  0.848214  15.437500  ...  0.917969  NaN  767972800  10221200   
2000-01-07  0.888393  16.187500  ...  0.854167  NaN  460734400   8253200   

                                                          
                 AMZN GOOG      MSFT NFLX      NVDA TSLA  
Date                                                      
2000-01-03  322352000  NaN  53228400  NaN  30091200  NaN  
2000-01-04  349748000  NaN  54119000  NaN  30048000  NaN  
2000-01-05  769148000  NaN  64059600  NaN  18835200  NaN  
2000-01-06  375040000  NaN  54976600  NaN  12048000  NaN  
2000-01-07  210108000  NaN  62013600  NaN   7118400  NaN  

[5 rows x 48 columns]

Pongamos las tres situaciones posibles como ejercicio:

1. Del DataFrame descargado, muestra el `Open` de todos los activos.
2. Del DataFrame descargado, muestra el `Close` de `'MSFT'`.
3. Del DataFrame descargado, muestra todos los atributos de `'NVDA'`.

In [13]:
# 1. Del DataFrame descargado, muestra el `Open` de todos los activos.

# SOLUCION
solucion_1 = df_yahoo['Open']
solucion_1.head()

,AAPL,ADBE,AMZN,GOOG,MSFT,NFLX,NVDA,TSLA
Date,,,,,,,,
2000-01-03,0.936384,16.812500,4.075000,NaN,58.68750,NaN,0.984375,NaN
2000-01-04,0.966518,15.750000,4.268750,NaN,56.78125,NaN,0.958333,NaN
2000-01-05,0.926339,14.562500,3.525000,NaN,55.56250,NaN,0.921875,NaN
2000-01-06,0.947545,15.359375,3.565625,NaN,56.09375,NaN,0.917969,NaN
2000-01-07,0.861607,15.390625,3.350000,NaN,54.31250,NaN,0.854167,NaN


In [14]:
# 2. Del DataFrame descargado, muestra el `Close` de 'MSFT'.

# SOLUCION
solucion_2 = df_yahoo['Close', 'MSFT']
solucion_2.head()

Date
2000-01-03    58.28125
2000-01-04    56.31250
2000-01-05    56.90625
2000-01-06    55.00000
2000-01-07    55.71875
Name: (Close, MSFT), dtype: float64

In [15]:
# 3. Del DataFrame descargado, muestra todos los atributos de 'NVDA'.

# SOLUCION
L = [x for x in df_yahoo.columns if x[1] == 'NVDA']
solucion_3 = df_yahoo[L]
solucion_3.columns = solucion_3.columns.droplevel(1) # Prueba a comentar esta línea. ¿Qué sucede?
solucion_3.head()

,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2000-01-03,0.894650,0.975260,0.992188,0.919271,0.984375,30091200
2000-01-04,0.870762,0.949219,0.960938,0.901042,0.958333,30048000
2000-01-05,0.842094,0.917969,0.937500,0.904948,0.921875,18835200
2000-01-06,0.787149,0.858073,0.917969,0.822917,0.917969,12048000
2000-01-07,0.800288,0.872396,0.881510,0.841146,0.854167,7118400


<hr>

#### 2. Descargarse datos de Quandl

`quandl` es una plataforma que proporciona acceso a una amplia gama de datos financieros y económicos de diferentes fuentes. La librería Quandl para Python ofrece una interfaz sencilla para acceder a estos datos desde la plataforma. Al igual que `yfinance`, esta librería es directamente compatible con `pandas`, ya que devuelve los datos en formato DataFrame.

Para instalar las liberías si fuese necesario:

```
pip install quandl

```

In [16]:
import quandl as ql

In [17]:
### PREGUNTA: ¿Cómo saber qué hay que poner en dataset para descargar?
### PREGUNTA: ¿Se pueden descargar varios activos como con yfinance?
### PREGUNTA: ¿Qué significan ex-dividend y split ratio?

In [23]:
# Descargamos los datos de Apple ('WIKI/AAPL') entre los años 2000 y 2010

df_quandl = ql.get(dataset='WIKI/AAPL',
                   start_date='2000-01-01', # Se indica con el argumento 'start_date' una fecha de incio (incluida)
                   end_date='2010-12-31') # Se indica con el argumento 'end_date' una fecha fin (OJO! incluida!)

La variable `df_quandl` es un DataFrame de Pandas con la siguiente información:

* Como índice del DataFrame, **Date**: La fecha a la que se recoge la información
* **Open**: Precio de apertura del activo
* **High**: Precio máximo alcanzado por el activo
* **Low**: Precio mínimo alcanzado por el activo
* **Close**: Precio de cierre del activo
* **Volume**: Número total de transacciones negociadas
* **Ex-Dividend**: ??
* **Split Ratio**: ??
* **Adj. Open**: Precio de apertura ajustado por eventos corporativos, como dividendos, splits, etc.
* **Adj. High**: Precio máximo ajustado por eventos corporativos, como dividendos, splits, etc.
* **Adj. Low**: Precio mínimo ajustado por eventos corporativos, como dividendos, splits, etc.
* **Adj. Close**: Precio de cierre ajustado por eventos corporativos, como dividendos, splits, etc.
* **Adj. Volume**: Número total de transacciones ajustadas

Veamos el aspecto que tiene:

In [24]:
df_quandl.head()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2000-01-03,104.87,112.50,101.69,111.94,4783900.0,0.0,1.0,3.369314,3.614454,3.267146,3.596463,133949200.0
2000-01-04,108.25,110.62,101.19,102.50,4574800.0,0.0,1.0,3.477908,3.554053,3.251081,3.293170,128094400.0
2000-01-05,103.75,110.56,103.00,104.00,6949300.0,0.0,1.0,3.333330,3.552125,3.309234,3.341362,194580400.0
2000-01-06,106.12,107.00,95.00,95.00,6856900.0,0.0,1.0,3.409475,3.437748,3.052206,3.052206,191993200.0
2000-01-07,96.50,101.00,95.50,99.50,4113700.0,0.0,1.0,3.100399,3.244977,3.068270,3.196784,115183600.0


In [25]:
df_quandl.tail()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2010-12-27,322.8519,325.44,321.52,324.68,8922000.0,0.0,1.0,41.490969,41.823576,41.319801,41.725905,62454000.0
2010-12-28,325.9102,326.66,325.06,325.47,6283000.0,0.0,1.0,41.884003,41.980363,41.774741,41.827431,43981000.0
2010-12-29,326.2200,326.45,325.10,325.29,5826400.0,0.0,1.0,41.923817,41.953375,41.779881,41.804299,40784800.0
2010-12-30,325.4800,325.51,323.05,323.66,5624800.0,0.0,1.0,41.828716,41.832572,41.516428,41.594821,39373600.0
2010-12-31,322.9500,323.48,321.31,322.56,6911000.0,0.0,1.0,41.503576,41.571689,41.292813,41.453456,48377000.0


In [26]:
df_quandl.shape # 2767 filas, 12 columnas

(2767, 12)

A partir de aquí, la metodología de trabajo es equivalente a trabajar con los datos de `yfinance` como en el apartado anterior. Sin embargo, hay una diferencia respecto a descargar un conjunto de activos:

In [27]:
# Descargamos los datos de Apple ('WIKI/AAPL') y Microsoft ('WIKI/MSFT') entre los años 2000 y 2010

activos = ["WIKI/AAPL", "WIKI/MSFT"]

df_quandl = ql.get(dataset=activos,
                   start_date='2000-01-01', # Se indica con el argumento 'start_date' una fecha de incio (incluida)
                   end_date='2010-12-31') # Se indica con el argumento 'end_date' una fecha fin (OJO! incluida!)

Aunque la descarga de los datos es equivalente a `yfinance`, en este caso no se genera un DataFrame multinivel en las columnas como antes. Con la librería Quandl, se define una columna por cada `activo - atributo`, veámoslo:

In [28]:
df_quandl.head()

,WIKI/AAPL - Open,WIKI/AAPL - High,WIKI/AAPL - Low,WIKI/AAPL - Close,WIKI/AAPL - Volume,WIKI/AAPL - Ex-Dividend,WIKI/AAPL - Split Ratio,WIKI/AAPL - Adj. Open,WIKI/AAPL - Adj. High,WIKI/AAPL - Adj. Low,...,WIKI/MSFT - Low,WIKI/MSFT - Close,WIKI/MSFT - Volume,WIKI/MSFT - Ex-Dividend,WIKI/MSFT - Split Ratio,WIKI/MSFT - Adj. Open,WIKI/MSFT - Adj. High,WIKI/MSFT - Adj. Low,WIKI/MSFT - Adj. Close,WIKI/MSFT - Adj. Volume
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,104.87,112.50,101.69,111.94,4783900.0,0.0,1.0,3.369314,3.614454,3.267146,...,112.00,116.56,26614200.0,0.0,1.0,39.066089,39.482146,37.278708,38.796484,53228400.0
2000-01-04,108.25,110.62,101.19,102.50,4574800.0,0.0,1.0,3.477908,3.554053,3.251081,...,112.25,112.62,27059500.0,0.0,1.0,37.797947,38.982878,37.361920,37.485073,54119000.0
2000-01-05,103.75,110.56,103.00,104.00,6949300.0,0.0,1.0,3.333330,3.552125,3.309234,...,109.37,113.81,32029800.0,0.0,1.0,36.985804,38.733244,36.403324,37.881159,64059600.0
2000-01-06,106.12,107.00,95.00,95.00,6856900.0,0.0,1.0,3.409475,3.437748,3.052206,...,108.37,110.00,27488300.0,0.0,1.0,37.341949,37.901130,36.070479,36.613017,54976600.0
2000-01-07,96.50,101.00,95.50,99.50,4113700.0,0.0,1.0,3.100399,3.244977,3.068270,...,107.31,111.44,31006800.0,0.0,1.0,36.153690,37.361920,35.717662,37.092315,62013600.0


Pongamos las tres situaciones posibles como ejercicio:

1. Del DataFrame descargado, muestra el `Open` de todos los activos.
2. Del DataFrame descargado, muestra el `Close` de `'MSFT'`.
3. Del DataFrame descargado, muestra todos los atributos de `'AAPL'`.

In [31]:
# 1. Del DataFrame descargado, muestra el `Open` de todos los activos.

# SOLUCION
L = [x for x in df_quandl.columns if "Open" in x and "Adj." not in x]
solucion_1 = df_quandl[L]
solucion_1.head()

,WIKI/AAPL - Open,WIKI/MSFT - Open
Date,,
2000-01-03,104.87,117.37
2000-01-04,108.25,113.56
2000-01-05,103.75,111.12
2000-01-06,106.12,112.19
2000-01-07,96.50,108.62


In [32]:
# 2. Del DataFrame descargado, muestra el `Close` de 'MSFT'.

# SOLUCION
solucion_2 = df_quandl["WIKI/MSFT - Close"]
solucion_2.head()

Date
2000-01-03    116.56
2000-01-04    112.62
2000-01-05    113.81
2000-01-06    110.00
2000-01-07    111.44
Name: WIKI/MSFT - Close, dtype: float64

In [33]:
# 3. Del DataFrame descargado, muestra todos los atributos de 'AAPL'.

# SOLUCION
L = [x for x in df_quandl.columns if "AAPL" in x]
solucion_3 = df_quandl[L]
solucion_3.head()

,WIKI/AAPL - Open,WIKI/AAPL - High,WIKI/AAPL - Low,WIKI/AAPL - Close,WIKI/AAPL - Volume,WIKI/AAPL - Ex-Dividend,WIKI/AAPL - Split Ratio,WIKI/AAPL - Adj. Open,WIKI/AAPL - Adj. High,WIKI/AAPL - Adj. Low,WIKI/AAPL - Adj. Close,WIKI/AAPL - Adj. Volume
Date,,,,,,,,,,,,
2000-01-03,104.87,112.50,101.69,111.94,4783900.0,0.0,1.0,3.369314,3.614454,3.267146,3.596463,133949200.0
2000-01-04,108.25,110.62,101.19,102.50,4574800.0,0.0,1.0,3.477908,3.554053,3.251081,3.293170,128094400.0
2000-01-05,103.75,110.56,103.00,104.00,6949300.0,0.0,1.0,3.333330,3.552125,3.309234,3.341362,194580400.0
2000-01-06,106.12,107.00,95.00,95.00,6856900.0,0.0,1.0,3.409475,3.437748,3.052206,3.052206,191993200.0
2000-01-07,96.50,101.00,95.50,99.50,4113700.0,0.0,1.0,3.100399,3.244977,3.068270,3.196784,115183600.0


<hr>

#### 3. Descargarse datos de AlphaVantage

AlphaVantage es una plataforma que proporciona datos financieros y servicios relacionados con el mercado. En lugar de utilizar un paquete de python, esta herramienta ofrece una API REST (Interfaz de Programación de Aplicaciones) que permite a los desarrolladores acceder mediante el protocolo HTTP a los datos, sea cual sea el lenguaje de programación. Para poder lanzar peticiones HTTP con Python necesitamos la librería `requests`.

In [49]:
import requests

In [ ]:
######################################################
#### ESTO POR AHORA NO ES NECESARIO. ¿BORRAR?
######################################################

# Specify the path to the file containing your API key
api_key_file = 'credentials/alphavantage_key'

# Try to open the file and read the API key
try:
    with open(api_key_file, 'r') as file:
        alpha_key = file.read().strip()  # Read the content and remove leading/trailing whitespace

    # Check if the API key is not empty
    if alpha_key:
        print("Alpha Key: Ha funcionado")
    else:
        print("The API key file is empty. Please check the file content.")

except FileNotFoundError:
    print(f"The file '{api_key_file}' was not found. Please make sure the file exists.")
except Exception as e:
    print(f"An error occurred while reading the API key: {str(e)}")


Como la mayoría de APIs REST, tenemos que preparar la URL de forma correcta para hacer la consulta y se devuelve la información solicitada en formato JSON. Veamos estos dos pasos poco a poco:

**Preparación de la URL para consultar a la API:**

Lo primero que necesitamos es acceder a la documentación de la API (https://www.alphavantage.co/documentation/) por si necesitamos algún parámetro adicional. Concretamente, vamos a utilizar la función **TIME_SERIES_INTRADAY** para descargar los datos de **Apple** con un **intervalo de 5 minutos**. Para ello definimos por trozos la URL:

In [41]:
function = "function=TIME_SERIES_INTRADAY"
symbol = "symbol=AAPL"
interval = "interval=5min"
key = "apikey=alpha_key"

Ahora tenemos que generar la URL completa:

In [47]:
URL = "https://www.alphavantage.co/query?"
URL = URL + '&'.join([function, symbol, interval, key])
print(URL)

https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=AAPL&interval=5min&apikey=alpha_key


¿Habéis probado a entrar en el enlace de arriba? Así es como muestra el explorador (Chrome, Firefox, o el que utilicéis) los datos JSON.

Ahora vamos a hacer la petición con la librería `requests`:

In [48]:
data_5min = requests.get(URL).json()
data_5min = r_5min.json()

print(data_5min)

{'Meta Data': {'1. Information': 'Intraday (5min) open, high, low, close prices and volume', '2. Symbol': 'AAPL', '3. Last Refreshed': '2024-01-10 19:55:00', '4. Interval': '5min', '5. Output Size': 'Full size', '6. Time Zone': 'US/Eastern'}, 'Time Series (5min)': {'2024-01-10 19:55:00': {'1. open': '186.6000', '2. high': '186.7000', '3. low': '186.4200', '4. close': '186.6200', '5. volume': '9033'}, '2024-01-10 19:50:00': {'1. open': '186.6000', '2. high': '186.7000', '3. low': '186.6000', '4. close': '186.6100', '5. volume': '1819'}, '2024-01-10 19:45:00': {'1. open': '186.4800', '2. high': '186.6800', '3. low': '186.4600', '4. close': '186.6000', '5. volume': '7635'}, '2024-01-10 19:40:00': {'1. open': '186.4700', '2. high': '186.5100', '3. low': '186.4200', '4. close': '186.4500', '5. volume': '2504'}, '2024-01-10 19:35:00': {'1. open': '186.4700', '2. high': '186.4900', '3. low': '186.4100', '4. close': '186.4700', '5. volume': '1322'}, '2024-01-10 19:30:00': {'1. open': '186.4000

Por supuesto, los datos están en formato JSON puro, por lo que es difícil interpretarlos. Vamos a transformar el JSON en un DataFrame de Pandas:

In [51]:
# Convierte el JSON en un DataFrame de Pandas
df_5min = pd.DataFrame.from_dict(data_5min['Time Series (5min)'], orient='index')

# Cambia los nombres de las columnas para eliminar los números y los puntos
df_5min.columns = [col.split('. ')[1] for col in df_5min.columns] # Prueba a comentar esta línea

df_5min.head()

,open,high,low,close,volume
2024-01-10 19:55:00,186.6000,186.7000,186.4200,186.6200,9033
2024-01-10 19:50:00,186.6000,186.7000,186.6000,186.6100,1819
2024-01-10 19:45:00,186.4800,186.6800,186.4600,186.6000,7635
2024-01-10 19:40:00,186.4700,186.5100,186.4200,186.4500,2504
2024-01-10 19:35:00,186.4700,186.4900,186.4100,186.4700,1322


A partir de aquí, podemos trabajar normalmente con los datos como hemos hecho con `yfinance` y `quandl`.

Hagamos ahora otro ejemplo para obtener los datos diarios. Simplemente llamamos a la función **TIME_SERIES_DAILY**:

In [53]:
function = "function=TIME_SERIES_DAILY" 
symbol = "symbol=AAPL"
key = "apikey=alpha_key"

URL = "https://www.alphavantage.co/query?"
URL = URL + '&'.join([function, symbol, key])
print(URL)

https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=AAPL&apikey=alpha_key


In [55]:
data_daily = requests.get(URL).json()
df_daily = pd.DataFrame.from_dict(data_daily['Time Series (Daily)'], orient='index')
df_daily.columns = [col.split('. ')[1] for col in df_daily.columns]
df_daily.head()

,open,high,low,close,volume
2024-01-10,184.3500,186.4000,183.9200,186.1900,46792908
2024-01-09,183.9200,185.1500,182.7300,185.1400,42841809
2024-01-08,182.0850,185.6000,181.5000,185.5600,59144470
2024-01-05,181.9900,182.7600,180.1700,181.1800,62196924
2024-01-04,182.1500,183.0872,180.8800,181.9100,71983570


Hagamos un último ejemplo para obtener información sobre Apple llamando a la función **SYMBOL_SEARCH**:

In [70]:
function = "function=SYMBOL_SEARCH" 
symbol = "keywords=Apple"
key = "apikey=alpha_key"

URL = "https://www.alphavantage.co/query?"
URL = URL + '&'.join([function, symbol, key])
print(URL)

https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords=Apple&apikey=alpha_key


In [85]:
nombre = 'Apple'
# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords=Apple&apikey=alpha_key'
r = requests.get(url)
data = r.json()

df_info = pd.DataFrame.from_dict(data['bestMatches'])
df_info.columns = [col.split('. ')[1] for col in df_info.columns]
df_info

,symbol,name,type,region,marketOpen,marketClose,timezone,currency,matchScore
0,APLE,Apple Hospitality REIT Inc,Equity,United States,09:30,16:00,UTC-04,USD,0.8889
1,AAPL,Apple Inc,Equity,United States,09:30,16:00,UTC-04,USD,0.7143
2,AAPL34.SAO,Apple Inc,Equity,Brazil/Sao Paolo,10:00,17:30,UTC-03,BRL,0.7143
3,APC.DEX,Apple Inc,Equity,XETRA,08:00,20:00,UTC+02,EUR,0.7143
4,APC.FRK,Apple Inc,Equity,Frankfurt,08:00,20:00,UTC+02,EUR,0.7143
5,AGPL,Apple Green Holding Inc,Equity,United States,09:30,16:00,UTC-04,USD,0.6667
6,0R2V.LON,Apple Inc.,Equity,United Kingdom,08:00,16:30,UTC+01,USD,0.6667
7,500014.BSE,Apple Finance Limited,Equity,India/Bombay,09:15,15:30,UTC+5.5,INR,0.3846
8,48T.FRK,APPLE HOSPITALITY REIT,Equity,Frankfurt,08:00,20:00,UTC+02,EUR,0.3704
9,603020.SHH,Apple Flavor Fragrance Group Company Ltd,Equity,Shanghai,09:30,15:00,UTC+08,CNY,0.2222


<hr>

In [59]:
##### REVISAR: ESTE CODIGO NO FUNCIONA. Earnings per Share EPS ya no está implementado en YF

In [60]:
# Definir el símbolo de Apple
symbol = 'AAPL'

# Crear un objeto Ticker para Apple
apple = yf.Ticker(symbol)

# Descargar los datos financieros
financial_data = apple.history(period="5y")

# Descargar datos de balance
balance_sheet = apple.balance_sheet.iloc[0]  # Utiliza el último balance disponible

# Calcular algunos ratios financieros
financial_data['Earnings per Share (EPS)'] = apple.earnings['Earnings'].iloc[-1]
financial_data['Price to Earnings Ratio (P/E)'] = financial_data['Close'] / financial_data['Earnings per Share (EPS)']
financial_data['Dividend Yield'] = apple.dividends['Dividends'].iloc[-1] / financial_data['Close']
financial_data['Price to Sales Ratio (P/S)'] = financial_data['Close'] / apple.info['trailingPE']
financial_data['Price to Book Ratio (P/B)'] = financial_data['Close'] / apple.info['bookValue']

# Ratios de Liquidez
financial_data['Current Ratio'] = balance_sheet['Total Current Assets'] / balance_sheet['Total Current Liabilities']
financial_data['Quick Ratio'] = (balance_sheet['Total Current Assets'] - balance_sheet['Inventory']) / balance_sheet['Total Current Liabilities']

# Ratios de Apalancamiento
financial_data['Debt to Equity Ratio'] = balance_sheet['Total Liab'] / balance_sheet['Total Stockholder Equity']
financial_data['Interest Coverage Ratio'] = apple.earnings['Interest Expense'] / apple.earnings['Earnings']

# Ratios adicionales
financial_data['Debt Ratio'] = balance_sheet['Total Liab'] / balance_sheet['Total Assets']
financial_data['Solvency Ratio'] = balance_sheet['Total Assets'] / balance_sheet['Total Liab']
financial_data['Liquidity Ratio'] = balance_sheet['Total Current Assets'] / balance_sheet['Total Current Liabilities']
financial_data['Treasury Ratio'] = (balance_sheet['Total Current Assets'] - balance_sheet['Inventory']) / balance_sheet['Total Current Liabilities']
financial_data['Financial Leverage Ratio'] = (balance_sheet['Total Assets'] / balance_sheet['Total Stockholder Equity']) * (apple.earnings['Total Other Income (Expense)'].iloc[-1] / apple.earnings['Total Revenue'].iloc[-1])

# Crear un DataFrame con los principales datos y ratios
report_data = pd.DataFrame({
    'Company': apple.info['longName'],
    'Symbol': symbol,
    'Industry': apple.info['industry'],
    'Sector': apple.info['sector'],
    'Market Cap': apple.info['marketCap'],
    'Beta': apple.info['beta'],
    'Current Price': financial_data['Close'].iloc[-1],
    '52-Week High': financial_data['High'].max(),
    '52-Week Low': financial_data['Low'].min(),
    'EPS': financial_data['Earnings per Share (EPS)'].iloc[-1],
    'P/E Ratio': financial_data['Price to Earnings Ratio (P/E)'].iloc[-1],
    'Dividend Yield': financial_data['Dividend Yield'].iloc[-1],
    'P/S Ratio': financial_data['Price to Sales Ratio (P/S)'].iloc[-1],
    'P/B Ratio': financial_data['Price to Book Ratio (P/B)'].iloc[-1],
    'Current Ratio': financial_data['Current Ratio'].iloc[-1],
    'Quick Ratio': financial_data['Quick Ratio'].iloc[-1],
    'Debt to Equity Ratio': financial_data['Debt to Equity Ratio'].iloc[-1],
    'Interest Coverage Ratio': financial_data['Interest Coverage Ratio'].iloc[-1],
    'Debt Ratio': financial_data['Debt Ratio'].iloc[-1],
    'Solvency Ratio': financial_data['Solvency Ratio'].iloc[-1],
    'Liquidity Ratio': financial_data['Liquidity Ratio'].iloc[-1],
    'Treasury Ratio': financial_data['Treasury Ratio'].iloc[-1],
    'Financial Leverage Ratio': financial_data['Financial Leverage Ratio'].iloc[-1]
})

# Guardar el informe en un archivo CSV
report_data.to_csv('apple_report.csv', index=False)

# Imprimir el informe en la consola
print("Informe de Apple:")
print(report_data)

print("\nInforme guardado en 'apple_report.csv'")

YFNotImplementedError: Have not implemented fetching 'earnings' from Yahoo API

In [ ]:
apple_info = apple.info

for key, value in apple_info.items():
    print(f"{key}: {value}")